In [1]:
import refinitiv.data as rd
from refinitiv.data.content import historical_pricing as hp


In [3]:
"""
#Start Refinitiv Session
rd.open_session(app_key="2ad05903f7cb4f84b75e13735fb71abde98f7d85")
"""

'\n#Start Refinitiv Session\nrd.open_session(app_key="2ad05903f7cb4f84b75e13735fb71abde98f7d85")\n'

In [5]:
"""
#Define instruments and Date range
START_DATE = "2006-01-01"
END_DATE = "2025-06-30"

# Replace these placeholders with the actual Refinitiv RICs 
SPX_PROXY_RIC     = "SPY.P"   # S&P 500 ETF (proxy for .SPX)
DOLLAR_INDEX_RIC  = ".DXY"    # Dollar Index
WTI_OIL_RIC       = "CLc1"    # WTI Crude Oil Futures (continuous contract)
"""

'\n#Define instruments and Date range\nSTART_DATE = "2006-01-01"\nEND_DATE = "2025-06-30"\n\n# Replace these placeholders with the actual Refinitiv RICs \nSPX_PROXY_RIC     = "SPY.P"   # S&P 500 ETF (proxy for .SPX)\nDOLLAR_INDEX_RIC  = ".DXY"    # Dollar Index\nWTI_OIL_RIC       = "CLc1"    # WTI Crude Oil Futures (continuous contract)\n'

In [7]:
#import refinitiv.data as rd
#print("Refinitiv Data Library version:", rd.__version__)

#print(dir(hp))

"""
from refinitiv.data.content import pricing

# request latest snapshot for Apple
snap = pricing.Definition(universe="AAPL.O").get_data()
print(snap)
"""

"""
#Obtain required data
# --- 1. S&P 500 OHLCV ---
df_spx_raw = hp.summaries.Definition(
    universe=SPX_PROXY_RIC,
    fields=["OPEN", "HIGH", "LOW", "CLOSE", "VOLUME"],
    interval=hp.Intervals.DAILY,
    start=START_DATE,
    end=END_DATE
).get_data()

# --- 2. Dollar Index & WTI Oil (Closing Prices) ---
df_macro_raw = hp.summaries.Definition(
    universe=[DOLLAR_INDEX_RIC, WTI_OIL_RIC],
    fields=["CLOSE"],
    interval=hp.Intervals.DAILY,
    start=START_DATE,
    end=END_DATE
).get_data()

# Output checks
print("S&P 500 Raw DataFrame (df_spx_raw):")
print(df_spx_raw.head())

print("\nMacro Raw DataFrame (df_macro_raw):")
print(df_macro_raw.head())
"""

'\n#Obtain required data\n# --- 1. S&P 500 OHLCV ---\ndf_spx_raw = hp.summaries.Definition(\n    universe=SPX_PROXY_RIC,\n    fields=["OPEN", "HIGH", "LOW", "CLOSE", "VOLUME"],\n    interval=hp.Intervals.DAILY,\n    start=START_DATE,\n    end=END_DATE\n).get_data()\n\n# --- 2. Dollar Index & WTI Oil (Closing Prices) ---\ndf_macro_raw = hp.summaries.Definition(\n    universe=[DOLLAR_INDEX_RIC, WTI_OIL_RIC],\n    fields=["CLOSE"],\n    interval=hp.Intervals.DAILY,\n    start=START_DATE,\n    end=END_DATE\n).get_data()\n\n# Output checks\nprint("S&P 500 Raw DataFrame (df_spx_raw):")\nprint(df_spx_raw.head())\n\nprint("\nMacro Raw DataFrame (df_macro_raw):")\nprint(df_macro_raw.head())\n'

In [ ]:
#Stationarity checks
#SPY log returns
adf_result = adfuller(df['SPY_log_ret'])
print(f"ADF Statistic: {adf_result[0]:.3f}, p-value: {adf_result[1]:.5f}")
if adf_result[1] < 0.05:
    print("✅ SPY_log_ret is stationary.")
else:
    print("⚠️ SPY_log_ret may not be stationary; differencing might be needed.")


#SPY log returns
adf_result_1 = adfuller(df['DXY_ret'])
print(f"ADF Statistic: {adf_result[0]:.3f}, p-value: {adf_result[1]:.5f}")
if adf_result[1] < 0.05:
    print("✅ DXY_ret is stationary.")
else:
    print("⚠️ DXY_ret may not be stationary; differencing might be needed.")


#SPY log returns
adf_result_2 = adfuller(df['WTI_ret'])
print(f"ADF Statistic: {adf_result[0]:.3f}, p-value: {adf_result[1]:.5f}")
if adf_result[1] < 0.05:
    print("✅ WTI_ret is stationary.")
else:
    print("⚠️ WTI_ret may not be stationary; differencing might be needed.")

In [ ]:
#Scale numerical features
features_to_scale = ['SPY_log_ret', 'SPY_p_change', 'DXY_ret', 'WTI_ret']
scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[features_to_scale] = scaler.fit_transform(df_scaled[features_to_scale])

#Check output
print("\nScaled feature sample:")
print(df_scaled[features_to_scale].head())

# Adaptive Wavelet Transform (AWT) Denoising & Decomposition

In [ ]:
#Import required libraries 
import pywt
import matplotlib.pyplot as plt

#1. Extract target signal
signal = df_scaled['SPY_log_ret'].values  # denoised target variable

#2. Choose wavelet parameters
wavelet = 'db4'         # Daubechies 4 is widely used in financial time series
level = 2               # decomposition level (2–4 typical for daily data)

#3. Perform discrete wavelet decomposition
coeffs = pywt.wavedec(signal, wavelet, level=level)

# where coeffs[0] = Approximation (trend), and
# coeffs[1:] = Detail coefficients (short-term fluctuations)

#4. Reconstruct the approximation and detail signals
approximation = pywt.waverec([coeffs[0]] + [None]*level, wavelet)
detail = signal - approximation[:len(signal)]  # ensure same length

# Clip both to the same index as df_scaled
approximation = approximation[:len(df_scaled)]
detail = detail[:len(df_scaled)]

#5. Add components to your DataFrame
df_scaled['SPY_s'] = approximation
df_scaled['SPY_d'] = detail

#6. Visual check of output
plt.figure(figsize=(12,6))
plt.plot(df_scaled.index, df_scaled['SPY_log_ret'], label='Original', alpha=0.6)
plt.plot(df_scaled.index, df_scaled['SPY_s'], label='Approximation (Trend)', color='orange')
plt.plot(df_scaled.index, df_scaled['SPY_d'], label='Detail (Fluctuation)', color='green', alpha=0.5)
plt.legend()
plt.title("AWT Decomposition of SPY_log_ret using Daubechies-4")
plt.show()

#7. Save the decomposed dataset
df_scaled.to_csv("03_awt_decomposed_data.csv")
print("\n✅ AWT decomposition complete. Data saved as '03_awt_decomposed_data.csv'")

# Feature Engineering: 21 Financial Technical Indicators

In [ ]:
# ================================================================
# DATA PREPARATION & MERGE (for yfinance-style Data)
# ================================================================

#Import required libraries
import re
import ta  # technical analysis library

# --- 1️⃣ Flatten MultiIndex columns if any ---
def flatten_columns(df):
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join([str(c) for c in col if c]) for col in df.columns]
    df.columns = [str(c).strip() for c in df.columns]
    return df

df_spx_full = flatten_columns(df_spx_raw)
df_scaled = flatten_columns(df_scaled)

# --- 2️⃣ Identify & rename likely OHLCV columns ---
ohlcv_candidates = [c for c in df_spx_full.columns if any(x in c.lower() for x in ['open', 'high', 'low', 'close', 'volume'])]
rename_map = {}
for col in ohlcv_candidates:
    lc = col.lower()
    if 'open' in lc: rename_map[col] = 'SPY_Open'
    elif 'high' in lc: rename_map[col] = 'SPY_High'
    elif 'low' in lc: rename_map[col] = 'SPY_Low'
    elif 'close' in lc and 'adj' not in lc: rename_map[col] = 'SPY_Close'
    elif 'adj' in lc and 'close' in lc: rename_map[col] = 'SPY_Adj_Close'
    elif 'volume' in lc: rename_map[col] = 'SPY_Volume'

df_spx_full = df_spx_full.rename(columns=rename_map)

# --- 3️⃣ Drop duplicates and merge safely ---
df_scaled = df_scaled.drop(columns=[c for c in ['SPY_Close'] if c in df_scaled.columns], errors='ignore')
df_feat = df_scaled.join(
    df_spx_full[[c for c in ['SPY_Open', 'SPY_High', 'SPY_Low', 'SPY_Close', 'SPY_Volume'] if c in df_spx_full.columns]],
    how='inner',
    rsuffix='_orig'
)

# --- 4️⃣ Clean up any whitespace or weird naming ---
df_feat.columns = df_feat.columns.map(lambda x: re.sub(r'\s+', '', str(x)))

# --- 5️⃣ Confirm required columns exist ---
required_cols = ['SPY_Open', 'SPY_High', 'SPY_Low', 'SPY_Close', 'SPY_Volume']
missing = [c for c in required_cols if c not in df_feat.columns]
if missing:
    raise KeyError(f"Missing SPY OHLCV columns: {missing}")
else:
    print("✅ All SPY OHLCV columns present.")

print(f"Final df_feat shape before feature generation: {df_feat.shape}")
print(df_feat.columns.tolist())


In [ ]:
# ================================================================
# FEATURE ENGINEERING: 21 TECHNICAL INDICATORS (CLEAN VERSION)
# ================================================================

import pandas as pd
import numpy as np
import ta  # technical analysis library

# ------------------------------------------------
# 0️⃣ Ensure clean, unique index
# ------------------------------------------------
df_feat = df_feat.copy()
df_feat = df_feat[~df_feat.index.duplicated(keep='first')]
df_feat = df_feat.sort_index()
print(f"✅ Index cleaned. Unique rows: {len(df_feat)}")

# ------------------------------------------------
# 1️⃣ Confirm required columns
# ------------------------------------------------
required_cols = ['SPY_Open', 'SPY_High', 'SPY_Low', 'SPY_Close', 'SPY_Volume', 'SPY_s']
for c in required_cols:
    if c not in df_feat.columns:
        raise KeyError(f"Missing required column: {c}")

print("✅ All required columns confirmed present.\n")

# ------------------------------------------------
# 2️⃣ Moving Averages (Trend)
# ------------------------------------------------
price_col = 'SPY_s'
vol_col = 'SPY_Volume'

df_feat['EMA20'] = ta.trend.EMAIndicator(df_feat[price_col], window=20).ema_indicator()
df_feat['MA5'] = ta.trend.SMAIndicator(df_feat[price_col], window=5).sma_indicator()
df_feat['MA10'] = ta.trend.SMAIndicator(df_feat[price_col], window=10).sma_indicator()
df_feat['V_MA5'] = ta.trend.SMAIndicator(df_feat[vol_col], window=5).sma_indicator()
df_feat['V_MA10'] = ta.trend.SMAIndicator(df_feat[vol_col], window=10).sma_indicator()

# ------------------------------------------------
# 3️⃣ Momentum Indicators
# ------------------------------------------------
df_feat['MACD'] = ta.trend.MACD(df_feat[price_col]).macd()
df_feat['ROC'] = ta.momentum.ROCIndicator(df_feat[price_col], window=10).roc()
df_feat['MTM6'] = df_feat[price_col] / df_feat[price_col].shift(126) - 1
df_feat['MTM12'] = df_feat[price_col] / df_feat[price_col].shift(252) - 1

stoch = ta.momentum.StochasticOscillator(
    high=df_feat['SPY_High'],
    low=df_feat['SPY_Low'],
    close=df_feat[price_col],
    window=14,
    smooth_window=3
)
df_feat['SMI'] = stoch.stoch_signal()

# ------------------------------------------------
# 4️⃣ Volatility & Pattern Indicators
# ------------------------------------------------
df_feat['ATR'] = ta.volatility.AverageTrueRange(
    high=df_feat['SPY_High'],
    low=df_feat['SPY_Low'],
    close=df_feat[price_col],
    window=14
).average_true_range()

boll = ta.volatility.BollingerBands(df_feat[price_col], window=20, window_dev=2)
df_feat['BOLL_upper'] = boll.bollinger_hband()
df_feat['BOLL_lower'] = boll.bollinger_lband()
df_feat['BOLL_bandwidth'] = df_feat['BOLL_upper'] - df_feat['BOLL_lower']

df_feat['CCI'] = ta.trend.CCIIndicator(
    high=df_feat['SPY_High'],
    low=df_feat['SPY_Low'],
    close=df_feat[price_col],
    window=20
).cci()

# ------------------------------------------------
# 5️⃣ Volume-based Indicator (WVAD)
# ------------------------------------------------
# Ensure alignment and valid numeric dtype
for c in ['SPY_Open', 'SPY_High', 'SPY_Low', 'SPY_Close', 'SPY_Volume']:
    df_feat[c] = pd.to_numeric(df_feat[c], errors='coerce')

df_feat['WVAD'] = (
    (df_feat['SPY_Close'] - df_feat['SPY_Open']) /
    (df_feat['SPY_High'] - df_feat['SPY_Low'] + 1e-10)
) * df_feat['SPY_Volume']

# ------------------------------------------------
# 6️⃣ Final Cleanup
# ------------------------------------------------
df_feat = df_feat.replace([np.inf, -np.inf], np.nan).dropna()
print(f"✅ Technical indicators successfully generated! Final shape: {df_feat.shape}")

# ------------------------------------------------
# 7️⃣ Save to file
# ------------------------------------------------
df_feat.to_csv("04_feature_engineered_data.csv")
print("💾 Saved as '04_feature_engineered_data.csv'")


In [ ]:
# 3️⃣ OPTIONAL VISUAL CHECKS
# ================================================================

# --- Plot MACD ---
plt.figure(figsize=(12, 5))
plt.plot(df_feat.index, df_feat[price_col], label='SPY_s (Smoothed Price)', color='black', alpha=0.7)
plt.plot(df_feat.index, df_feat['EMA20'], label='EMA20', linestyle='--')
plt.title("Price with EMA20")
plt.legend()
plt.show()

plt.figure(figsize=(12, 5))
plt.plot(df_feat.index, df_feat['MACD'], label='MACD', color='blue')
plt.axhline(0, color='gray', linestyle='--', linewidth=1)
plt.title("MACD Indicator")
plt.legend()
plt.show()

# --- Plot Bollinger Bands ---
plt.figure(figsize=(12, 5))
plt.plot(df_feat.index, df_feat[price_col], label='SPY_s (Smoothed Price)', color='black')
plt.plot(df_feat.index, df_feat['BOLL_upper'], label='Bollinger Upper', linestyle='--', color='red')
plt.plot(df_feat.index, df_feat['BOLL_lower'], label='Bollinger Lower', linestyle='--', color='green')
plt.fill_between(df_feat.index, df_feat['BOLL_lower'], df_feat['BOLL_upper'], color='gray', alpha=0.1)
plt.title("Bollinger Bands")
plt.legend()
plt.show()

# --- Plot Volume & WVAD ---
plt.figure(figsize=(12, 4))
plt.plot(df_feat.index, df_feat['SPY_Volume'], label='SPY Volume', color='orange', alpha=0.6)
plt.plot(df_feat.index, df_feat['WVAD'], label='WVAD', color='purple', alpha=0.8)
plt.title("Volume and WVAD")
plt.legend()
plt.show()

In [ ]:
# ================================================================
# TWO-PANEL VERIFICATION PLOT: Price + Bollinger (top), MACD (bottom)
# ================================================================
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

# --- Safety: ensure needed columns are present ---
_need = ['SPY_s', 'BOLL_upper', 'BOLL_lower', 'MACD']
_missing = [c for c in _need if c not in df_feat.columns]
if _missing:
    raise KeyError(f"Missing columns for plot: {_missing}")

# Optional: focus window (e.g., last 2 years). Comment out to use full history.
# dfp = df_feat.last('730D')  # last ~730 days
dfp = df_feat.copy()

fig = plt.figure(figsize=(14, 8))
gs = fig.add_gridspec(nrows=2, ncols=1, height_ratios=[3, 1.4], hspace=0.08)

# -------------------------
# Top panel: Price + Bands
# -------------------------
ax_price = fig.add_subplot(gs[0, 0])
ax_price.plot(dfp.index, dfp['SPY_s'], label='SPY_s (Smoothed Price)')
ax_price.plot(dfp.index, dfp['BOLL_upper'], label='Bollinger Upper', linestyle='--')
ax_price.plot(dfp.index, dfp['BOLL_lower'], label='Bollinger Lower', linestyle='--')
ax_price.fill_between(dfp.index, dfp['BOLL_lower'], dfp['BOLL_upper'], alpha=0.10)
ax_price.set_title("SPY_s with Bollinger Bands")
ax_price.legend(loc='upper left')
ax_price.grid(True, alpha=0.3)

# -------------------------
# Bottom panel: MACD
# -------------------------
ax_macd = fig.add_subplot(gs[1, 0], sharex=ax_price)
ax_macd.plot(dfp.index, dfp['MACD'], label='MACD')
ax_macd.axhline(0.0, linestyle='--', linewidth=1)
ax_macd.set_title("MACD")
ax_macd.legend(loc='upper left')
ax_macd.grid(True, alpha=0.3)

# Tidy x-axis formatting
date_fmt = DateFormatter("%Y-%m")
ax_macd.xaxis.set_major_formatter(date_fmt)
plt.setp(ax_price.get_xticklabels(), visible=False)

plt.show()

# Fit FIEGARCH(1,d,1)-t in R from Python

In [ ]:
# === 1) rpy2 + R sanity check (Windows-safe ABI mode) ===
# - Confirms rpy2 can see your R
# - Prints R version and home
# - (No pandas2ri.activate used — we rely on localconverter only)

import os
import rpy2.robjects as ro

# Optional: reduce thread contention before R loads (helps stability on Windows/Anaconda)
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

print("R version:", ro.r("R.version.string")[0])
print("R home   :", ro.r("R.home()")[0])

In [ ]:
import rpy2.robjects as ro

r = ro.r

# 1) Choose a writable user library and prepend it to .libPaths()
r('''
# Compute a per-user library path if R didn't define one
userlib <- Sys.getenv("R_LIBS_USER")
if (is.na(userlib) || userlib == "") {
  userlib <- file.path(Sys.getenv("USERPROFILE"), "R",
                       paste0("win-library"),
                       paste0(R.version$major, ".", R.version$minor))
}
dir.create(userlib, recursive = TRUE, showWarnings = FALSE)
.libPaths(c(userlib, .libPaths()))
''')

# 2) Avoid interactive questions & use Windows binaries when possible
r('options(repos="https://cloud.r-project.org", pkgType="win.binary", ask=FALSE)')

# 3) Install prerequisites into the user library (explicit lib= to avoid prompts)
r('install.packages(c("xts","zoo","Rcpp","RcppArmadillo","numDeriv","Rsolnp"), '
  'lib=.libPaths()[1])')

# 4) Install rugarch
r('install.packages("rugarch", lib=.libPaths()[1])')

# 5) Verify
print("R home   :", r('R.home()')[0])
print("Lib paths:", list(r('.libPaths()')))
print("rugarch  :", r('as.character(packageVersion("rugarch"))')[0])

# Converters + quick rugarch import check

In [6]:
# === 2) Converters + quick 'rugarch' import check ===
# - Defines to_r / from_r conversion helpers (no global activate)
# - Verifies that 'rugarch' loads inside your R

from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

def to_r(obj):
    """Python -> R with pandas DataFrame/Series support (scoped)."""
    with localconverter(ro.default_converter + pandas2ri.converter):
        return ro.conversion.py2rpy(obj)

def from_r(obj):
    """R -> Python with pandas DataFrame/Series support (scoped)."""
    with localconverter(ro.default_converter + pandas2ri.converter):
        return ro.conversion.rpy2py(obj)

# Smoke-load 'rugarch' so we fail early if it's missing
ro.r('suppressMessages(library(rugarch))')
print("rugarch loaded OK")

Error importing in API mode: ImportError('On Windows, cffi mode "ANY" is only "ABI".')
Trying to import in ABI mode.


NameError: name 'ro' is not defined

# Call FIEGARCH via R (rugarch)

In [ ]:
# === 3) FIEGARCH(1,d,1)-t via R (rugarch) callable from Python ===
# Fit FIEGARCH on ARIMAX residuals. If the FIEGARCH spec fails, it will
# gracefully fall back to FIGARCH(1,d,1) so you always get a result.
#
# Inputs
#   resid   : pd.Series (ARIMAX residuals, indexed)
#   h_ahead : int, forecast horizon (e.g., len(val_df))
#   var_exog: optional pd.DataFrame of variance exogenous regressors
#             (columns like ['DXY_ret','WTI_ret']) aligned to resid.index
#   dist    : "std" (Student-t), "norm", "ged", ...
#
# Returns dict:
#   aic, bic, loglik, d, gamma (may be NaN if not estimated), nu,
#   var_fc (pd.Series of sigma^2 forecasts length h_ahead),
#   std_resid (pd.Series, standardized resid on the input window),
#   coef (pd.Series of R coefficient vector),
#   used_model ("FIEGARCH" or "FIGARCH"), r_warnings (list of strings)

import numpy as np
import pandas as pd

def fit_fiegarch_r(resid, h_ahead, var_exog=None, dist="std"):
    # ---- prep inputs
    y = pd.Series(resid).dropna().astype(float)
    ro.globalenv.clear()
    ro.globalenv["y"] = to_r(y.to_frame("y"))

    if isinstance(var_exog, pd.DataFrame):
        X = var_exog.loc[y.index].astype(float)
        ro.globalenv["X"] = to_r(X)
    else:
        ro.globalenv["X"] = ro.NULL

    ro.globalenv["h"]    = ro.IntVector([int(h_ahead)])
    ro.globalenv["dist"] = ro.StrVector([dist])

    # ---- R code (tries FIEGARCH first; if it errors, tries FIGARCH)
    r_code = """
    suppressMessages(library(rugarch))

    yy   <- y$y
    Xmat <- if (is.null(X)) NULL else as.matrix(X)

    mk_spec <- function(use_fiegarch = TRUE) {
      if (use_fiegarch) {
        ugarchspec(
          variance.model = list(model="fiGARCH", submodel="FIEGARCH",
                                garchOrder=c(1,1), external.regressors=Xmat),
          mean.model     = list(armaOrder=c(0,0), include.mean=FALSE),
          distribution.model = dist
        )
      } else {
        ugarchspec(
          variance.model = list(model="fiGARCH", submodel="FIGARCH",
                                garchOrder=c(1,1), external.regressors=Xmat),
          mean.model     = list(armaOrder=c(0,0), include.mean=FALSE),
          distribution.model = dist
        )
      }
    }

    # Try FIEGARCH, then fall back to FIGARCH on error
    warn_buf <- character(0)
    withCallingHandlers({
      spec_try <- try(mk_spec(TRUE), silent=TRUE)
      if (inherits(spec_try, "try-error")) {
        spec <- mk_spec(FALSE)
        used <- "FIGARCH"
      } else {
        spec <- spec_try
        used <- "FIEGARCH"
      }
      fit <- ugarchfit(spec = spec, data = yy, solver = "hybrid")
      fc  <- ugarchforecast(fit, n.ahead = h[1])
    },
    warning = function(w) { warn_buf <<- c(warn_buf, conditionMessage(w)); invokeRestart("muffleWarning") }
    )

    cf  <- coef(fit); nms <- names(cf)

    # pick out fractional d, leverage gamma (if present), and nu/shape
    d_idx  <- grep("delta|longMem|d\\[", nms, ignore.case=TRUE)
    g_idx  <- grep("^gamma", nms, ignore.case=TRUE)
    nu_idx <- grep("^shape$|^nu$|^shape1$", nms, ignore.case=TRUE)

    d_val  <- if (length(d_idx)>0)  cf[d_idx[1]] else NA_real_
    g_val  <- if (length(g_idx)>0)  cf[g_idx[1]] else NA_real_
    nu_val <- if (length(nu_idx)>0) cf[nu_idx[1]] else NA_real_

    list(
      aic       = infocriteria(fit)[1],
      bic       = infocriteria(fit)[2],
      loglik    = likelihood(fit),
      d         = as.numeric(d_val),
      gamma     = as.numeric(g_val),
      nu        = as.numeric(nu_val),
      sigma2_fc = as.numeric(sigma(fc)^2),
      std_resid = as.numeric(residuals(fit, standardize=TRUE)),
      coef      = cf,
      used      = used,
      warnings  = warn_buf
    )
    """

    res = ro.r(r_code)

    # ---- pull back results
    aic   = float(res.rx2("aic")[0])
    bic   = float(res.rx2("bic")[0])
    ll    = float(res.rx2("loglik")[0])
    d_hat = float(res.rx2("d")[0])      if len(res.rx2("d"))      else np.nan
    gamma = float(res.rx2("gamma")[0])  if len(res.rx2("gamma"))  else np.nan
    nu    = float(res.rx2("nu")[0])     if len(res.rx2("nu"))     else np.nan
    used  = str(res.rx2("used")[0])     if len(res.rx2("used"))   else "FIEGARCH"

    var_fc    = pd.Series(from_r(res.rx2("sigma2_fc")), name="var_fc_fi")
    std_resid = pd.Series(from_r(res.rx2("std_resid")), index=y.index, name="std_resid_fi")

    coef_r = res.rx2("coef")
    coef   = pd.Series(from_r(coef_r), index=list(coef_r.names))

    r_warnings = list(from_r(res.rx2("warnings"))) if len(res.rx2("warnings")) else []

    return dict(aic=aic, bic=bic, loglik=ll, d=d_hat, gamma=gamma, nu=nu,
                var_fc=var_fc, std_resid=std_resid, coef=coef,
                used_model=used, r_warnings=r_warnings)

# Rebuild ARIMAX residuals + define val_df (fixed order, auto column-finder)

In [ ]:
# === 4) Rebuild ARIMAX residuals + define val_df (fixed order 2,0,3) ===
# Requirements in df (either existing returns or price columns we can turn to returns):
#   - SPY log returns  -> 'SPY_log_ret'  (else inferred from price like 'SPY_Close')
#   - Dollar index ret -> 'DXY_ret'      (else inferred from price like 'DXY_Close' or 'DX-Y.NYB')
#   - WTI oil returns  -> 'WTI_ret'      (else inferred from price like 'WTI_Close' or 'CL=F')

import re
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

assert 'df' in globals(), "I need your dataframe `df` in memory."
df = df.copy()

# ---- 4.1 find likely price columns (robust, case-insensitive)
def find_candidates(df, include_any, include_close=True, top_n=8):
    kws = [k.lower() for k in include_any]
    price_kws = ['close','adj','last','px','price']
    scored = []
    for c in df.columns:
        cl = str(c).lower()
        # basic token match
        name_score = sum(1 for k in kws if k in cl)
        if name_score == 0:
            continue
        if include_close and not any(pk in cl for pk in price_kws):
            continue
        # extra points for exact-ish patterns
        if re.search(r'\bspy\b', cl): name_score += 2
        if re.search(r'\bdxy\b', cl): name_score += 2
        if 'cl=f' in cl:             name_score += 2
        scored.append((name_score, c))
    scored.sort(key=lambda x: (-x[0], str(x[1])))
    return [c for _, c in scored[:top_n]]

spy_price_cands = find_candidates(df, include_any=['spy','s&p','s and p','sp500'])
dxy_price_cands = find_candidates(df, include_any=['dxy','dx-y','usdx','dollar index','us dollar'])
wti_price_cands = find_candidates(df, include_any=['wti','cl=f','crude','oil','wti crude'])

print("\nDetected price column candidates:")
print("SPY:", spy_price_cands or "(none)")
print("DXY:", dxy_price_cands or "(none)")
print("WTI:", wti_price_cands or "(none)")

# Manual overrides (set a column name here if auto-pick is not what you want)
MANUAL_SPY = None  # e.g. "SPY_Close"
MANUAL_DXY = None  # e.g. "DXY_Close" or "DX-Y.NYB"
MANUAL_WTI = None  # e.g. "CL=F" or "WTI_Close"

c_spy_px = MANUAL_SPY or (spy_price_cands[0] if spy_price_cands else None)
c_dxy_px = MANUAL_DXY or (dxy_price_cands[0] if dxy_price_cands else None)
c_wti_px = MANUAL_WTI or (wti_price_cands[0] if wti_price_cands else None)

# ---- 4.2 ensure we have (or build) SPY_log_ret, DXY_ret, WTI_ret
def make_log_ret(series):
    s = pd.to_numeric(series, errors='coerce')
    return np.log(s / s.shift(1)) * 100

def ensure_return(df, out_col, ret_aliases, px_col, must=True):
    # use an existing column if any alias is present
    for al in ret_aliases:
        if al in df.columns:
            df[out_col] = pd.to_numeric(df[al], errors='coerce')
            return df
    # else build from price column
    if px_col is not None and px_col in df.columns:
        df[out_col] = make_log_ret(df[px_col])
        return df
    if must:
        raise AssertionError(
            f"Could not find or build {out_col}. "
            f"Set MANUAL_* to a valid price column or verify df columns."
        )
    return df

spy_ret_alias = ['SPY_log_ret','SPY_ret','ret_spy','y_1d','y_t','SPY_Returns']
dxy_ret_alias = ['DXY_ret','DX_ret','DXY_Returns','Dollar_ret','USDX_ret']
wti_ret_alias = ['WTI_ret','CL_ret','WTI_Returns','OIL_ret','Crude_ret']

df = ensure_return(df, 'SPY_log_ret', spy_ret_alias, c_spy_px, must=True)
df = ensure_return(df, 'DXY_ret',     dxy_ret_alias, c_dxy_px, must=True)
df = ensure_return(df, 'WTI_ret',     wti_ret_alias, c_wti_px, must=True)

print("\nHead of constructed/confirmed returns:")
print(df[['SPY_log_ret','DXY_ret','WTI_ret']].dropna().head())

# ---- 4.3 split and fit ARIMAX(2,0,3) on train, keep residuals for volatility model
df = df.sort_index()
try:
    df.index = pd.to_datetime(df.index)
except Exception:
    pass

n = len(df)
i_tr = int(n*0.8); i_va = int(n*0.9)
train_df = df.iloc[:i_tr].copy()
val_df   = df.iloc[i_tr:i_va].copy()
test_df  = df.iloc[i_va:].copy()

y_tr = train_df['SPY_log_ret'].astype(float)
X_tr = train_df[['DXY_ret','WTI_ret']].astype(float)

order = (2, 0, 3)
sm_model = SARIMAX(y_tr, exog=X_tr, order=order,
                   enforce_stationarity=False, enforce_invertibility=False)
sm_res = sm_model.fit(disp=False)

resid_for_vol = pd.Series(sm_res.resid, index=y_tr.index, name='resid_tr')
arimax_out = {'order': order, 'aic': sm_res.aic, 'resid_tr': resid_for_vol}

print(f"\nARIMAX (fixed) order: {order} | AIC: {sm_res.aic:.2f} | resid shape: {resid_for_vol.shape}")
print("train/val/test ranges:",
      f"train {train_df.index.min()} → {train_df.index.max()} |",
      f"val {val_df.index.min()} → {val_df.index.max()} |",
      f"test {test_df.index.min()} → {test_df.index.max()}")

# Quick Smoke Test: Verify FIEGARCH Pipeline (rugarch via rpy2)

In [11]:
# === 5) Quick Smoke Test: verify R→Python FIEGARCH integration ===
# Purpose:
#   - Confirm Python↔R bridge works
#   - Confirm rugarch FIEGARCH spec compiles
#   - Confirm ugarchfit() completes on synthetic data

import numpy as np
import pandas as pd

# Tiny synthetic test sample
y = pd.Series(np.random.randn(200))

# Register in R environment
ro.globalenv.clear()
ro.globalenv['y'] = to_r(y.to_frame('y'))

# Run minimal FIEGARCH spec (fiGARCH/FIEGARCH)
print("Running FIEGARCH smoke test via rugarch...")

ro.r('''
suppressMessages(library(rugarch))
yy <- y$y

spec <- ugarchspec(
  variance.model = list(model="fiGARCH", submodel="FIEGARCH",
                        garchOrder=c(1,1)),
  mean.model     = list(armaOrder=c(0,0), include.mean=FALSE),
  distribution.model = "std"
)

fit <- ugarchfit(spec=spec, data=yy, solver="hybrid")
print(fit)
''')

print("✅ FIEGARCH smoke test completed successfully.")

NameError: name 'ro' is not defined

# Full FIEGARCH Fit Call (using ARIMAX residuals)

In [ ]:
# === 6) ULTRA-SAFE FIEGARCH(1,d,1)-t fit using ARIMAX residuals (FIGARCH fallback) ===
# Run this right after the smoke test. If it’s stable, you can relax the caps later.

import numpy as np
import pandas as pd
import rpy2.robjects as ro

# --- Keep R to 1 thread (inside R). We do NOT touch Python threads here to avoid clashes.
ro.r(r"""
suppressMessages({
  if (requireNamespace("RhpcBLASctl", quietly=TRUE)) {
    RhpcBLASctl::blas_set_num_threads(1)
    RhpcBLASctl::omp_set_num_threads(1)
  }
})
""")

# --- Prepare residuals: percent -> decimals, drop non-finite, soft-clip extreme tails
resid_full = pd.to_numeric(arimax_out['resid_tr'], errors='coerce').dropna() / 100.0
if not np.isfinite(resid_full).all():
    resid_full = resid_full[np.isfinite(resid_full)]

# soft clip to +/- 8 * std (rugarch can blow up with extreme outliers on first fit)
s = resid_full.std()
if s > 0:
    resid_full = resid_full.clip(lower=-8*s, upper=8*s)

# Start with a small window; increase after stability (e.g., 2000 -> 4000 -> 9000)
MAX_N = 2000
resid_dec = resid_full.iloc[-min(MAX_N, len(resid_full)) : ].copy()

# Forecast horizon: start small; you can switch to len(val_df) after stability
h_va_cap = 30
h_va = int(min(len(val_df), h_va_cap, max(1, len(resid_dec)//10)))  # simple guard

# No variance exogenous regressors on the first stable run
var_exog = None

# ---- Run the fit via your bridge, but ask it to return R warnings and fallback info.
# To get warnings & model used, we pass through via a slightly safer R wrapper.
r_code = """
function(y, h, X=NULL, dist="std") {
  suppressMessages(library(rugarch))
  set.seed(123)

  yy   <- as.numeric(y)
  Xmat <- if (is.null(X)) NULL else as.matrix(X)

  warns <- character()

  mk_spec <- function(use_fiegarch=TRUE) {
    if (use_fiegarch) {
      ugarchspec(
        variance.model = list(model="fGARCH", submodel="FIEGARCH",
                              garchOrder=c(1,1), external.regressors=Xmat),
        mean.model     = list(armaOrder=c(0,0), include.mean=FALSE),
        distribution.model = dist
      )
    } else {
      ugarchspec(
        variance.model = list(model="fGARCH", submodel="FIGARCH",
                              garchOrder=c(1,1), external.regressors=Xmat),
        mean.model     = list(armaOrder=c(0,0), include.mean=FALSE),
        distribution.model = dist
      )
    }
  }

  fit_once <- function(spec) {
    withCallingHandlers({
        fit <- ugarchfit(
          spec = spec, data = yy,
          solver = "hybrid",
          fit.control = list(scale = TRUE, stationarity = 1),
          solver.control = list(trace = 0)
        )
        fit
      },
      warning=function(w) { warns <<- c(warns, conditionMessage(w)); invokeRestart("muffleWarning") }
    )
  }

  # Try FIEGARCH first, then FIGARCH if needed
  used_model <- "FIEGARCH"
  fit <- try(fit_once(mk_spec(TRUE)), silent=TRUE)
  if (inherits(fit, "try-error")) {
    used_model <- "FIGARCH"
    fit <- fit_once(mk_spec(FALSE))
  }

  fc <- ugarchforecast(fit, n.ahead = h)

  cf  <- coef(fit); nms <- names(cf)
  d_idx  <- grep("delta|longMem|d\\[", nms, ignore.case=TRUE)
  g_idx  <- grep("^gamma", nms, ignore.case=TRUE)
  nu_idx <- grep("^shape$|^nu$|^shape1$", nms, ignore.case=TRUE)

  d_val  <- if (length(d_idx)>0)  cf[d_idx[1]] else NA_real_
  g_val  <- if (length(g_idx)>0)  cf[g_idx[1]] else NA_real_
  nu_val <- if (length(nu_idx)>0) cf[nu_idx[1]] else NA_real_

  list(
    aic       = infocriteria(fit)[1],
    bic       = infocriteria(fit)[2],
    loglik    = likelihood(fit),
    d         = as.numeric(d_val),
    gamma     = as.numeric(g_val),
    nu        = as.numeric(nu_val),
    sigma2_fc = as.numeric(sigma(fc)^2),
    std_resid = as.numeric(residuals(fit, standardize=TRUE)),
    coef      = cf,
    used_model= used_model,
    r_warnings= warns
  )
}
"""

# Build the R wrapper once
r_fit_wrapper = ro.r(r_code)

# Call it
from rpy2.robjects.conversion import localconverter
from rpy2.robjects import pandas2ri

with localconverter(ro.default_converter + pandas2ri.converter):
    r_y  = ro.conversion.py2rpy(resid_dec.to_frame("y"))
    r_X  = ro.NULL  # var_exog=None for now
    res  = r_fit_wrapper(r_y.rx2("y"), h_va, r_X, "std")

# Pull back results
def rvec_to_series(x, name=None, index=None):
    with localconverter(ro.default_converter + pandas2ri.converter):
        s = ro.conversion.rpy2py(x)
    if isinstance(s, pd.Series):
        if name is not None: s.name = name
        if index is not None and len(index) == len(s): s.index = index
        return s
    return pd.Series(s, index=index, name=name)

aic   = float(res.rx2("aic")[0])
bic   = float(res.rx2("bic")[0])
ll    = float(res.rx2("loglik")[0])
d_hat = float(res.rx2("d")[0]) if len(res.rx2("d")) else np.nan
gamma = float(res.rx2("gamma")[0]) if len(res.rx2("gamma")) else np.nan
nu    = float(res.rx2("nu")[0]) if len(res.rx2("nu")) else np.nan
used_model = str(res.rx2("used_model")[0])
warns = list(ro.vectors.ListVector(res).rx2("r_warnings"))

var_fc    = rvec_to_series(res.rx2("sigma2_fc"), name="var_fc_fiegarch")
std_resid = rvec_to_series(res.rx2("std_resid"), index=resid_dec.index, name="std_resid_fiegarch")

coef_r = res.rx2("coef")
with localconverter(ro.default_converter + pandas2ri.converter):
    coef = pd.Series(ro.conversion.rpy2py(coef_r), index=list(coef_r.names))

# Align variance forecast to validation index (short horizon for the very first run)
var_fc_fiegarch = var_fc.set_axis(val_df.index[:len(var_fc)])

print(f"\nModel used : {used_model}")
print(f"Sample     : {len(resid_dec)} (of total {len(resid_full)}) | h={len(var_fc)}")
print(f"AIC={aic:.3f}, BIC={bic:.3f}, logLik={ll:.2f}, d={d_hat:.3f}, gamma={gamma:.3f}, nu={nu:.2f}")
if warns:
    print("\n▲ R warnings:")
    for w in warns:
        print(" -", w)

# Pack for later steps
fiegarch_out = {
    "aic": aic, "bic": bic, "loglik": ll, "d": d_hat, "gamma": gamma, "nu": nu,
    "var_fc": var_fc_fiegarch, "std_resid": std_resid, "coef": coef, "used_model": used_model,
    "r_warnings": warns
}

print("\nVariance forecast (head):")
print(var_fc_fiegarch.head())

In [ ]:
# ============================================================
# Build & save FULL AWT + 21-indicator + exogenous dataset
# ============================================================
from pathlib import Path

BASE = Path("data/processed")
BASE.mkdir(parents=True, exist_ok=True)

# 1) Build SPY core AWT + indicators + y_1d from raw SPY OHLCV
#    Use the original Yahoo SPY frame, but flatten its MultiIndex columns.
df_spy_raw_flat = flatten_cols(df_spx_raw.copy())
print("Flattened SPY columns:", df_spy_raw_flat.columns)

df_awt_core = compute_awt_features(df_spy_raw_flat)
print("Core AWT shape (SPY only):", df_awt_core.shape)

# 2) Exogenous returns (DXY, WTI) from your merged & cleaned dataframe `df`
#    `df` is from the "Data Preprocessing & Transformation" section.
#    Align them explicitly to the SPY index.
df_exog = df[['DXY_ret', 'WTI_ret']].copy()
df_exog = df_exog.reindex(df_awt_core.index)

print("Exog columns:", df_exog.columns)
print("Exog shape (reindexed):", df_exog.shape)

# 3) Combine core AWT + exogenous on the Date index
#    Use concat (same effect as a left join on the index).
df_awt = pd.concat([df_awt_core, df_exog], axis=1)

# Only require that the target exists; keep rows even if some indicators/exog have NaNs
df_awt = df_awt[df_awt['y_1d'].notna()]

print("Final AWT + exogenous + target shape:", df_awt.shape)

# 4) Save
out_path = BASE / "df_awt_features.csv"
df_awt.to_csv(out_path)
print("✅ Saved AWT + 21 indicators + exogenous + target to:", out_path.resolve())
print(df_awt.head())

In [ ]:
# ================================================================
# 3️⃣  Compute AWT + Features + Target
# ================================================================
# ------------------------------------------------
# Robust AWT + features + target (auto-make SPY_log_ret if missing)
# ------------------------------------------------
def _first_present(colnames, df_cols):
    for c in colnames:
        if c in df_cols:
            return c
    return None

def compute_awt_features(df_slice: pd.DataFrame) -> pd.DataFrame:
    """
    Build AWT-based features + 21 indicators safely.
    Requires SPY OHLCV if available; will fall back where possible.
    Produces:
      SPY_s, SPY_d, and 21 indicators incl. MACD, RSI, ADX, OBV, MFI, WVAD, etc.
    """
    # -------- helpers --------
    def _flatten(df):
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = ['_'.join([str(c) for c in tup if c is not None]).strip()
                          for tup in df.columns]
        else:
            df.columns = [str(c).strip() for c in df.columns]
        return df.loc[:, ~pd.Index(df.columns).duplicated()]

    def _find_col(df, *must_contain):
        must = [m.lower() for m in must_contain]
        for c in df.columns:
            lc = str(c).lower()
            if all(m in lc for m in must):
                return c
        # simple fallbacks for common single-level names
        lut = {
            ('spy', 'open'):   ['SPY_Open', 'Open', 'Open_SPY'],
            ('spy', 'high'):   ['SPY_High', 'High', 'High_SPY'],
            ('spy', 'low'):    ['SPY_Low', 'Low', 'Low_SPY'],
            ('spy', 'close'):  ['SPY_Close', 'Close', 'Close_SPY'],
            ('spy', 'volume'): ['SPY_Volume', 'Volume', 'Volume_SPY'],
        }
        for k, alts in lut.items():
            if set(k) == set(must):
                for a in alts:
                    if a in df.columns:
                        return a
        return None

    def _wavelet_denoise(x, wavelet="db2", level=2, mode="soft"):
        x = pd.Series(x).astype(float)
        coeffs = pywt.wavedec(x.values, wavelet=wavelet, level=level)
        sigma = np.median(np.abs(coeffs[-1])) / 0.6745 if len(coeffs[-1]) else 0.0
        thr   = sigma * np.sqrt(2 * np.log(len(x)))
        coeffs_thr = [coeffs[0]] + [pywt.threshold(c, thr, mode=mode) for c in coeffs[1:]]
        x_s = pywt.waverec(coeffs_thr, wavelet)[:len(x)]
        x_d = x.values[:len(x_s)] - x_s
        return pd.DataFrame({"SPY_s": x_s, "SPY_d": x_d}, index=x.index)

    # -------- start --------
    df_f = _flatten(df_slice.copy())

    # Locate OHLCV columns (best effort)
    c_open   = _find_col(df_f, 'spy', 'open')
    c_high   = _find_col(df_f, 'spy', 'high')
    c_low    = _find_col(df_f, 'spy', 'low')
    c_close  = _find_col(df_f, 'spy', 'close')
    c_volume = _find_col(df_f, 'spy', 'volume')

    # Ensure log returns exist (needed for AWT)
    if 'SPY_log_ret' not in df_f.columns:
        if c_close is None:
            raise KeyError("compute_awt_features() needs SPY close prices to derive SPY_log_ret.")
        df_f['SPY_log_ret'] = np.log(df_f[c_close] / df_f[c_close].shift(1)) * 100

    # AWT on SPY_log_ret -> SPY_s (smooth), SPY_d (detail)
    s_hat = _wavelet_denoise(df_f['SPY_log_ret'])
    df_f = df_f.join(s_hat, how='left')

    # Choose price series for TA (prefer denoised)
    price = df_f['SPY_s'].fillna(df_f['SPY_log_ret'])

    # ---------------- 21 INDICATORS ----------------
    # 1–3 Trend MAs
    df_f['EMA20'] = ta.trend.EMAIndicator(close=price, window=20).ema_indicator()
    df_f['MA5']   = ta.trend.SMAIndicator(close=price, window=5).sma_indicator()
    df_f['MA10']  = ta.trend.SMAIndicator(close=price, window=10).sma_indicator()

    # 4 MACD
    df_f['MACD']  = ta.trend.MACD(close=price).macd()

    # 5 ROC(10)
    df_f['ROC']   = ta.momentum.ROCIndicator(close=price, window=10).roc()

    # 6–7 Momentum (6m, 12m)
    df_f['MTM6']  = price / price.shift(126) - 1
    df_f['MTM12'] = price / price.shift(252) - 1

    # 8 SMI (stochastic momentum index)
    if c_high and c_low:
        stoch = ta.momentum.StochasticOscillator(high=df_f[c_high], low=df_f[c_low],
                                                 close=price, window=14, smooth_window=3)
        df_f['SMI'] = stoch.stoch_signal()
    else:
        df_f['SMI'] = np.nan

    # 9 ATR(14)
    df_f['ATR'] = (ta.volatility.AverageTrueRange(
        high=df_f[c_high] if c_high else price,
        low=df_f[c_low] if c_low else price,
        close=price, window=14
    ).average_true_range())

    # 10–12 Bollinger
    boll = ta.volatility.BollingerBands(close=price, window=20, window_dev=2)
    df_f['BOLL_upper']     = boll.bollinger_hband()
    df_f['BOLL_lower']     = boll.bollinger_lband()
    df_f['BOLL_bandwidth'] = df_f['BOLL_upper'] - df_f['BOLL_lower']

    # 13 CCI(20)
    if c_high and c_low and c_close:
        df_f['CCI'] = ta.trend.CCIIndicator(
            high=df_f[c_high], low=df_f[c_low], close=df_f[c_close], window=20
        ).cci()
    else:
        df_f['CCI'] = np.nan

    # 14 WVAD (needs OHLCV)
    if c_open and c_high and c_low and c_close and c_volume:
        rng = (df_f[c_high] - df_f[c_low]).replace(0, np.nan)
        df_f['WVAD'] = ((df_f[c_close] - df_f[c_open]) / (rng + 1e-10)) * df_f[c_volume]
    else:
        df_f['WVAD'] = np.nan

    # 15 RSI(14)
    df_f['RSI14'] = ta.momentum.RSIIndicator(close=price, window=14).rsi()

    # 16–17 Stochastic %K/%D (classic)
    if c_high and c_low:
        st = ta.momentum.StochasticOscillator(high=df_f[c_high], low=df_f[c_low],
                                              close=price, window=14, smooth_window=3)
        df_f['STO_K'] = st.stoch()
        df_f['STO_D'] = st.stoch_signal()
    else:
        df_f[['STO_K', 'STO_D']] = np.nan

    # 18 ADX(14)
    if c_high and c_low and c_close:
        df_f['ADX14'] = ta.trend.ADXIndicator(
            high=df_f[c_high], low=df_f[c_low], close=df_f[c_close], window=14
        ).adx()
    else:
        df_f['ADX14'] = np.nan

    # 19 OBV
    if c_close and c_volume:
        df_f['OBV'] = ta.volume.OnBalanceVolumeIndicator(
            close=df_f[c_close], volume=df_f[c_volume]
        ).on_balance_volume()
    else:
        df_f['OBV'] = np.nan

    # 20 MFI(14)
    if c_high and c_low and c_close and c_volume:
        df_f['MFI14'] = ta.volume.MFIIndicator(
            high=df_f[c_high], low=df_f[c_low],
            close=df_f[c_close], volume=df_f[c_volume], window=14
        ).money_flow_index()
    else:
        df_f['MFI14'] = np.nan

    # 21 Williams %R(14)
    if c_high and c_low and c_close:
        df_f['WILLR14'] = ta.momentum.WilliamsRIndicator(
            high=df_f[c_high], low=df_f[c_low], close=df_f[c_close], lbp=14
        ).williams_r()
    else:
        df_f['WILLR14'] = np.nan

    # Target (example: next-day log return)
    df_f['y_1d'] = df_f['SPY_log_ret'].shift(-1)

    # --- Final clean ---
    # Only require that the target exists; keep rows even if some indicators have NaNs.
    # This avoids dropping the entire dataset.
    df_f = df_f[df_f['y_1d'].notna()]

    return df_f

In [ ]:
# ============================================================
# Rolling 1-step-ahead forecasts (ARIMAX mean + EGARCH variance)
# Using fixed EGARCH(2,1,1)-t and selected ARIMAX(p,d,q)
# ============================================================

from pathlib import Path
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from arch import arch_model

# --- inputs expected from previous cells:
# y, X  -> prepared series/dataframe
# p, d_, q  -> selected ARIMAX orders from Cell 4
ep, eo, eq, edist = 2, 1, 1, 't'  # fixed EGARCH config

START  = int(len(y) * 0.70)       # expanding window start (70% train)
OUTDIR = Path("data/processed"); OUTDIR.mkdir(parents=True, exist_ok=True)

mu_vals, mu_idx = [], []
var_vals, var_idx = [], []

# Optional: keep last ARIMAX params to speed convergence
last_params = None

for t in range(START, len(y)):
    y_tr = y.iloc[:t]
    X_tr = X.iloc[:t]
    X_fc = X.iloc[[t]]

    # --- ARIMAX mean forecast (1-step)
    mod = SARIMAX(y_tr, order=(p, d_, q), exog=X_tr, trend='n',
                  enforce_stationarity=False, enforce_invertibility=False)
    # warm-start if available
    res = mod.fit(disp=False, start_params=last_params) if last_params is not None else mod.fit(disp=False)
    last_params = res.params.values  # cache for next iteration

    mu_hat = res.get_forecast(steps=1, exog=X_fc).predicted_mean.iloc[0]
    mu_vals.append(mu_hat); mu_idx.append(y.index[t])

    # --- EGARCH variance forecast (1-step) on ARIMAX residuals
    resid_tr = res.resid.dropna()
    eg = arch_model(resid_tr, mean='Zero', vol='EGARCH',
                    p=ep, o=eo, q=eq, dist=edist).fit(disp='off')
    var_hat = eg.forecast(horizon=1, reindex=True).variance.iloc[-1, 0]
    var_vals.append(var_hat); var_idx.append(y.index[t])

# Build result series
mu_fc  = pd.Series(mu_vals,  index=pd.DatetimeIndex(mu_idx),  name='arimax_mu_fc1')
var_fc = pd.Series(var_vals, index=pd.DatetimeIndex(var_idx), name='egarch_var_fc1')

# Save artifacts
mu_fc.to_csv(OUTDIR / "arimax_mu_fc1.csv")
var_fc.to_csv(OUTDIR / "egarch_var_fc1.csv")

print("✅ Saved:",
      OUTDIR / "arimax_mu_fc1.csv",
      OUTDIR / "egarch_var_fc1.csv")
